In [6]:
# Import all the things
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.figure_factory as ff
import waterfall_chart
import altair as alt
from vega_datasets import data as vega_data
import plotly.plotly as py
import plotly.graph_objs as go
from itertools import product
import colorlover as cl


# General settings
pd.options.display.max_rows = 10
pd.options.display.max_seq_items = 200

static_dir = "slides/static/"

In [8]:
%store -r df

Where did Rep votes come from?


In [9]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')
    
colors = alt.Chart(counties).mark_geoshape().encode(
    color='per_gop_2016_100%:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['per_gop_2016_100%'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)
colors + outlines 
chart = colors + outlines 
chart.save('slides/static/rep_2016.html')

Where did Dem votes come from?

In [10]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')
    
colors = alt.Chart(counties).mark_geoshape().encode(
    color='per_dem_2016_100%:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['per_dem_2016_100%'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)
colors + outlines 
chart = colors + outlines 
chart.save('slides/static/dem_2016.html')

Where did Dem turnout change?

In [11]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')

domain = [df.min()['dem_vote_change'], 0, df.max()['dem_vote_change']]
range_ = ['darkred', 'white', 'green']
colors = alt.Chart(counties).mark_geoshape().encode(
    color=alt.Color('dem_vote_change:Q', scale=alt.Scale(domain=domain, range=range_, type='linear'))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['dem_vote_change'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

colors + outlines 
chart = colors + outlines 
chart.save('slides/static/dem_gain.html')

Where did rep turnout change

In [13]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')

domain = [df.min()['rep_vote_change'], 0, df.max()['rep_vote_change']]
range_ = ['darkred', 'white', 'green']
colors = alt.Chart(counties).mark_geoshape().encode(
    color=alt.Color('rep_vote_change:Q', scale=alt.Scale(domain=domain, range=range_, type='linear'))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['rep_vote_change'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

colors + outlines 
chart = colors + outlines 
chart.save('slides/static/rep_gain.html')

Where did counties trend more Republican?

In [14]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')

domain = [df.min()['republican_shift'], 0, df.max()['republican_shift']]
range_ = ['blue', 'white', 'red']
colors = alt.Chart(counties).mark_geoshape().encode(
    color=alt.Color('republican_shift:Q', scale=alt.Scale(domain=domain, range=range_, type='linear'))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['republican_shift'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

colors + outlines 
chart = colors + outlines 
chart.save('slides/static/rep_shift.html')

Where are the Mormon votes

In [15]:
counties = alt.topo_feature(vega_data.us_10m.url, 'counties')

states = alt.topo_feature(vega_data.us_10m.url, 'states')

outlines = alt.Chart(states).mark_geoshape(
    stroke='black'
).project('albersUsa')
    
colors = alt.Chart(counties).mark_geoshape().encode(
    color='LDSRATE:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df.dropna(subset=['LDSRATE']), 'id', ['LDSRATE'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)
colors + outlines 
chart = colors + outlines 
chart.save('slides/static/mormons.html')

Switching To Basic Scatters

In [ ]:
not_id_ut = df[(df.state_abbr != 'UT') & (df.state_abbr != 'ID')] 
id_ut = df[(df.state_abbr == 'UT') | (df.state_abbr == 'ID')] 
trace = go.Scatter(
    x=not_id_ut['High School Only or Less, 2012-2016'],
    y=not_id_ut['republican_shift'],
    text=not_id_ut['GEO.display-label'],
    mode = 'markers',
    name='NOT ID and UT',
)
trace2 = go.Scatter(
    x=id_ut['High School Only or Less, 2012-2016'],
    y=id_ut['republican_shift'],
    text=id_ut['GEO.display-label'],
    mode = 'markers',
    name='ID and UT',

)
layout = go.Layout(
    title='Republican Shift vs Education',
    xaxis=dict(title='High School Only or Less, 2012 through 2016'),
    yaxis=dict(title='Republican Presidential Vote %, 2016 minus 2012'),
)
fig = go.Figure(data=[trace, trace2],layout=layout)
plotly.offline.plot(fig, filename=static_dir+'education_vs_shift.html')


In [ ]:
not_id_ut = df[(df.state_abbr != 'UT') & (df.state_abbr != 'ID')] 
trace = go.Scatter(
    x=not_id_ut['High School Only or Less, 2012-2016'],
    y=not_id_ut['republican_shift'],
    text=not_id_ut['GEO.display-label'],
    mode = 'markers',
    name='NOT ID and UT',
)
layout = go.Layout(
    title='Republican Shift vs Education, No ID/UT',
    xaxis=dict(title='High School Only or Less, 2012 through 2016'),
    yaxis=dict(title='Republican Presidential Vote %, 2016 minus 2012'),
)
fig = go.Figure(data=[trace],layout=layout)
plotly.offline.plot(fig, filename=static_dir+'education_vs_shift_wo_id_ut.html')


In [ ]:
not_id_ut = df[(df.state_abbr != 'UT') & (df.state_abbr != 'ID')] 
trace = go.Scatter(
    x=not_id_ut['percent_white'],
    y=not_id_ut['republican_shift'],
    text=not_id_ut['GEO.display-label'],
    mode = 'markers',
    name='NOT ID and UT',
)
layout = go.Layout(
    title='Republican Shift vs County Race Composition, No ID/UT',
    xaxis=dict(title='Percent of County that is White'),
    yaxis=dict(title='Republican Presidential Vote %, 2016 minus 2012'),
)
fig = go.Figure(data=[trace],layout=layout)
plotly.offline.plot(fig, filename=static_dir+'race_vs_shift_wo_id_ut.html')


In [ ]:

cell_df = df[(df.state_abbr != 'UT') & (df.state_abbr != 'ID')& (df.total_votes_2016>1000)]
trace1 = go.Scatter3d(
    x=cell_df['percent_white'],
    y=cell_df['High School Only or Less, 2012-2016'],
    z=cell_df['republican_shift'],
    text=cell_df['GEO.display-label'],
    mode='markers',
    marker=dict(
        size=4,
        color=cell_df['Median_Household_Income_2016'],
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8,
    )
)
data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
   scene = dict(
    aspectmode='cube',
    xaxis = dict(title='% White'),
    yaxis = dict(title='% High School Only or Less, 2012-2016'),
    zaxis = dict(title='Republican Shift (%age Points)'),),       
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='slides/static/race_vs_ed_vs_shift.html')